In [ ]:
%pip install transformers

In [ ]:
from pathlib import Path
import os
!pip install gdown==v4.6.3
data_path = Path(os.getcwd()) # This one will return your current path, so it will works better
file_path = data_path / "cpsc_599_27_project_dataset.csv"

if file_path.exists():
  print("The file existed")
else:
  !gdown --id 17-Sqq_DCGMarHwBSqjclJflUfYtHN0a0 -O cpsc_599_27_project_dataset.csv
  # The link is https://drive.google.com/file/d/17-Sqq_DCGMarHwBSqjclJflUfYtHN0a0/view?usp=sharing

import pandas as pd
df = pd.read_csv("/content/cpsc_599_27_project_dataset.csv")

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=17-Sqq_DCGMarHwBSqjclJflUfYtHN0a0
To: /content/cpsc_599_27_project_dataset.csv
100% 197k/197k [00:00<00:00, 94.0MB/s]


In [ ]:
df

,Question,Answer
0,Who is Ho Chi Minh?,Ho Chi Minh was a Vietnamese Communist revolut...
1,When and where was Ho Chi Minh born?,"Ho Chi Minh was born on May 19, 1890, in Nghe ..."
2,What was Ho Chi Minh's original name?,Ho Chi Minh's original name was Nguyen Sinh Cung.
3,Describe Ho Chi Minh's role in the Vietnam War.,Ho Chi Minh played a crucial role in leading N...
4,What political ideology did Ho Chi Minh follow?,Ho Chi Minh followed Marxist-Leninist politica...
...,...,...
775,how did hồ chí minhs thought and revolution im...,His thought and revolution inspired many leade...
776,as a communist leader how was hồ chí minh perc...,"As a communist, he was one of the few internat..."
777,what did unesco recommend its member states do...,"In 1987, UNESCO officially recommended that it..."
778,what significant contributions of hồ chí minh ...,"One of Hồ Chí Minh's works, The Black Race, mu..."


In [ ]:
questions = df.Question
answers = df.Answer

In [ ]:
# Preprocessing step
questions = questions.replace("Hồ Chí Minh", "Ho Chi Minh")
questions = questions.replace("hồ chí minh", "ho chi minh")
questions = questions.replace("Hồ Chí Minh's", "Ho Chi Minh's")
questions = questions.replace("hồ chí minh's", "ho chi minh's")

answers = answers.replace("Hồ Chí Minh", "Ho Chi Minh")
answers = answers.replace("hồ chí minh", "ho chi minh")
answers = answers.replace("Hồ Chí Minh's", "Ho Chi Minh's")
answers = answers.replace("hồ chí minh's", "ho chi minh's")

In [ ]:
print(int(len(df)/2))

390


In [ ]:
questions_arr = []
answers_arr = []

for i in range(len(df)):
  questions_arr.append(questions[i])
  answers_arr.append(answers[i])

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, BertTokenizer
import torch

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


# Suppose you have a list of contexts and targets
contexts = ["Albert Einstein was a theoretical physicist.", "bababaab"]
targets = ["Who was Albert Einstein?", "ednsoiwed"]

# Tokenize the contexts and targets
# input_ids = tokenizer(contexts, return_tensors='pt', truncation=True, padding=True).input_ids
# labels = tokenizer(targets, return_tensors='pt', truncation=True, padding=True).input_ids
input_ids = tokenizer(answers_arr, return_tensors='pt', truncation=True, padding=True).input_ids
labels = tokenizer(questions_arr, return_tensors='pt', truncation=True, padding=True).input_ids

# Create a PyTorch dataset
dataset = torch.utils.data.TensorDataset(input_ids, labels)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

In [ ]:
# Initialize the model
model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)

# Set up the optimizer
optimizer = torch.optim.AdamW(model.parameters())

# Set up the training loop
for epoch in range(10):  # Number of epochs
    for batch in torch.utils.data.DataLoader(dataset, batch_size=32):  # Batch size
        optimizer.zero_grad()
        outputs = model(input_ids=batch[0].to(device), labels=batch[1].to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Suppose you have a new sentence
sentence = "In Ho Chi Minh: A Life, William Duiker considered this hypothesis, but ultimately rejected it."

# Tokenize the sentence
input_ids = tokenizer(sentence, return_tensors='pt').input_ids.to(device)

# Generate a question
output_ids = model.generate(input_ids)
question = tokenizer.decode(output_ids[0])

# Remove padding tokens and strip whitespace
question = question.replace('<pad>', '').strip()
question = question.replace('</s>', '').strip()
question = question.replace('h<unk>ch', 'Ho Chi Minh').strip() # Format error (as Hồ Chí Minh can't be encoded properly)
question = question.replace('H<unk> Ch<unk>Minh', 'Ho Chi Minh').strip()

# Capitalize the first letter
question = question[0].upper() + question[1:]

# Add a question mark at the end
if not question.endswith('?'):
    question += '?'

question

'What was william duikers stance on the hypothesis regarding Ho Chi Minh?'

In [ ]:
# Suppose you have a new sentence
sentence = "Ho Chi Minh was a Vietnamese Communist revolutionary leader who was the founding father and first President of the Democratic Republic of Vietnam (North Vietnam)."

# Tokenize the sentence
input_ids = tokenizer(sentence, return_tensors='pt').input_ids.to(device)

# Generate a question
output_ids = model.generate(input_ids)
question = tokenizer.decode(output_ids[0])

# Remove padding tokens and strip whitespace
question = question.replace('<pad>', '').strip()
question = question.replace('</s>', '').strip()
question = question.replace('h<unk>ch', 'Ho Chi Minh').strip() # Format error (as Hồ Chí Minh can't be encoded properly)
question = question.replace('H<unk> Ch<unk>Minh', 'Ho Chi Minh').strip()

# Capitalize the first letter
question = question[0].upper() + question[1:]

# Add a question mark at the end
if not question.endswith('?'):
    question += '?'

question

'Who is ho chi minh?'

In [ ]:
import numpy as np

# Australia, Austria, Belgium, Canada, Chile, Colombia, Costa_Rica, Czech, Denmark, Estonia,
# Finland, France, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Japan, South Korea, Latvia, Lithuania, Luxemborg,
# Mexico, Dutch, NZ, Norway, Poland, Slovakia, Slovenia, Spain, Sweden, Swit, Turkey, UK, US

# Let's assume you have the following x and y data points
x = np.array([42, 30, 37, 54, 36, 22, 18, 22, 36, 38, 42, 32, 27, 28, 23, 37, 41, 49, 17, 48, 45, 30, 27, 46, 19, 34, 36, 42, 27, 20, 29, 35, 39, 40, 17, 42, 44])
y = np.array([51, 36, 46, 63, 31, 28, 25, 27, 42, 42, 43, 42, 33, 35, 29, 44, 54, 52, 20, 56, 53, 39, 47, 51, 21, 45, 40, 48, 34, 29, 40, 41, 49, 45, 25, 51, 50])

# You can use numpy's corrcoef function to find the correlation matrix
correlation_matrix = np.corrcoef(x, y)

# The correlation between x and y will be at the positions [0][1] and [1][0] in the matrix
correlation_xy = correlation_matrix[0,1]

print(f"The correlation between x and y is {correlation_xy}")

The correlation between x and y is 0.9283546180825043


In [ ]:
answer = sum(y)/sum(x)
answer

1.2046362909672261

In [ ]:
# Brazil, China, Indo, Russia, Saudi
z = np.array([14, 17, 12, 54, 22])
answer_z = answer * z
answer_z

array([16.86490807, 20.47881695, 14.45563549, 65.05035971, 26.5019984 ])

Merge missing datapoint

In [ ]:
import pandas as pd

main_df = pd.read_csv("/content/Copy of Copy_of_merged_data_2022 - merged_data_2022.csv")
gdp_nominal_df = pd.read_csv("/content/GDP per capita (nominal).csv")
gdp_ppp_df = pd.read_csv("/content/GDP per capita (PPP).csv")
unemployment_df = pd.read_csv("/content/Unemployment rate.csv")

In [ ]:
gdp_nominal_df.iloc[:, [-1]]

,2028
0,NaN
1,no data
2,11187.728
3,5365.19
4,46920.949
...,...
994,NaN
995,NaN
996,NaN
997,NaN


In [ ]:
len(gdp_nominal_df)

999

In [ ]:
gdp_nominal_df.columns[-7]

'2022'

In [ ]:
# Merge the last column of the first two files
merged_df = pd.merge(main_df, gdp_nominal_df[['Country', gdp_nominal_df.columns[-7]]], on='Country', how='outer')

# Continue merging the last column of the remaining files
merged_df = pd.merge(merged_df, gdp_ppp_df[['Country', gdp_ppp_df.columns[-7]]], on='Country', how='outer')
merged_df = pd.merge(merged_df, unemployment_df[['Country', unemployment_df.columns[-7]]], on='Country', how='outer')

In [ ]:
merged_df.head(191)

,Unnamed: 0,HDI Components 2021 Rank,Country,HDI Components 2021 Human Development Index (HDI),Secondary Education 2022 Total,Infrastructure Index,HDI 2021,UN_PPP2022_Output_PopTot 2022,Global Happiness Index Life Ladder,GDP per capita (PPP) 2022,GDP per capita (nominal) 2022,Unemployment rate 2022,Real GDP Growth 2022,GDP per capita (nominal) 2028,GDP per capita (PPP) 2028,Unemployment rate 2028,2022_x,2022_y,2022
0,0.0,187.0,Burundi,0.426,4.0,NaN,0.426,12 890,NaN,855.814,310.986,NaN,5.5,309.158,1122.831,NaN,310.986,855.814,NaN
1,1.0,181.0,Sierra Leone,0.477,20.0,NaN,0.477,8 606,2.560,2009.191,479.777,NaN,4.6,500.631,2629.350,NaN,479.777,2009.191,NaN
2,2.0,188.0,Central African Republic,0.404,6.0,NaN,0.404,5 579,NaN,1081.194,490.940,NaN,3.3,672.103,1300.731,NaN,490.94,1081.194,NaN
3,3.0,173.0,Madagascar,0.501,5.0,NaN,0.501,29 612,4.019,1817.445,523.084,NaN,4.5,685.073,2302.592,NaN,523.084,1817.445,NaN
4,4.0,169.0,Malawi,0.512,11.0,NaN,0.512,20 405,3.356,1627.813,567.454,NaN,4.6,503.116,1950.966,NaN,567.454,1627.813,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,186.0,153.0,"Congo, Republic of",0.571,30.0,NaN,0.571,5 970,NaN,5276.499,2838.369,NaN,NaN,3358.965,6542.815,NaN,2838.369,5276.499,NaN
187,187.0,138.0,São Tomé and Príncipe,0.618,NaN,NaN,0.618,227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2413.087,4067.21,no data
188,188.0,45.0,Slovak Republic,0.848,93.0,78.6,0.848,5 643,NaN,40210.852,21263.247,NaN,NaN,32300.319,53339.435,5.9,21263.247,40210.852,6.2
189,189.0,191.0,"South Sudan, Republic of",0.385,NaN,NaN,0.385,10 913,NaN,456.346,585.003,NaN,NaN,522.262,583.660,NaN,585.003,456.346,NaN


In [ ]:
merged_df.head(191).to_csv("data_501_dataset.csv")